In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, roc_curve, auc

In [ ]:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica",
  "font.size": 14
})

def plot_ribbon(ax, x, y, s, color, alpha = 0.25, label = None):
  ax.fill_between(x, y - s, y + s, color = color,
                  edgecolor = None, alpha = alpha)
  ax.plot(x, y, color = color, label = label)

In [ ]:
paths_data = op.join("/path", "to", "data")
paths_save = op.join("paths", "to", "figure11")
os.makedirs(paths_save, exist_ok = True)

In [ ]:
dataset_list = ["multi-shell", "single-shell"]
method_list  = ["Original", "FWE", "MSMT"]
suffix_list  = ["ytrue", "ypred", "yprob"] 

conds = itertools.product(dataset_list, method_list, suffix_list)

df_boot = dict() # initialize
for dataset, method, suffix in conds: # for each condition
  fname = f"figure11_{dataset}_method-{method}_{suffix}.npy"
  df_boot[(dataset, method, suffix)] = np.load(op.join(paths_data, fname))

In [ ]:
# calculate averaged over repeats: MAE, Correlation, and R2
df_metrics = {} # initialize

for dataset in dataset_list: # for each dataset and tract
  print(f"Dataset: [{dataset}] ----------------------------------------------")
  for method in method_list: # for each method
    curr_key = (dataset, method)
    y_true = df_boot[curr_key + ("ytrue",)]
    y_pred = df_boot[curr_key + ("ypred",)]
    n_repeats = y_true.shape[1] # number of repeats
    
    mae = np.empty(n_repeats); corr = np.empty(n_repeats); r2 = np.empty(n_repeats)
    for i in np.arange(n_repeats): # for each repeat
      i_true, i_pred = y_true[:,i], y_pred[:,i] # current true and predicted values
      mae[i]  = np.mean(np.abs(i_true - i_pred)) # mean absolute error
      corr[i] = np.corrcoef(i_true, i_pred)[0,1] # correlation
      r2[i]   = r2_score(i_true, i_pred) # R2 score
    
    df_metrics[(dataset, method)] = {"mae": mae, "corr": corr, "r2": r2}

    stats_str = f"{method:9s}: MAE = {mae.mean():0.4f} ({mae.std():0.4f}), "
    stats_str += f"r = {corr.mean():0.4f} ({corr.std():0.4f}), "
    stats_str += f"R2 = {r2.mean():0.4f} ({r2.std():0.4f})"
    print(stats_str) # print statistics
  print()

In [ ]:
pair_list = [
  [(2), (3, 4, 5, 6)],
  [(2, 3), (4, 5, 6)], 
  [(2, 3, 4), (5, 6)], 
  [(2, 3, 4, 5), (6)], 
]

# false positive rate, resampled to 100 points
x_fpr = np.linspace(0, 1, 100)

df_prob = [] # initialize 
for i, (label_a, label_b) in enumerate(pair_list): # for each pair of fazekas scores  

  conds = itertools.product(dataset_list, method_list)
  for dataset, method in conds: # for each condition
    # get true labels, predicted probabilities, and unique labels
    curr_key = (dataset, method)
    y_true   = df_boot[curr_key + ("ytrue",)] # true labels
    y_prob   = df_boot[curr_key + ("yprob",)] # predictded probabilities
    y_labels = np.unique(y_true) # unique labels

    # get column indices of pair A and B
    indx_a = np.where(np.isin(y_labels, label_a))[0]
    indx_b = np.where(np.isin(y_labels, label_b))[0]

    # get predicted probabilities of pair A and B
    yprob_a = y_prob[:,indx_a,:].sum(axis = 1)
    yprob_b = y_prob[:,indx_b,:].sum(axis = 1)

    # get true labels for pair A and B (as float)
    ytrue_a = np.isin(y_true, label_a) * 1.0 
    ytrue_b = np.isin(y_true, label_b) * 1.0

    n_repeats = ytrue_a.shape[1] # number of repeats
    tpr_ab = np.empty((len(x_fpr), n_repeats)) # initialize
    auc_ab = np.empty(n_repeats) # initialize
    for i in np.arange(n_repeats): # for each repeat
      # calculate ROC curves and AUC for group A and B
      fpr_a, tpr_a, _ = roc_curve(ytrue_a[:,i], yprob_a[:,i]); auc_a = auc(fpr_a, tpr_a) 
      fpr_b, tpr_b, _ = roc_curve(ytrue_b[:,i], yprob_b[:,i]); auc_b = auc(fpr_b, tpr_b)

      # calulate the average between the two ROCs
      tpr_a = np.interp(x_fpr, fpr_a, tpr_a)
      tpr_b = np.interp(x_fpr, fpr_b, tpr_b)
      tpr_ab[:,i] = (tpr_a + tpr_b) / 2
      auc_ab[i] = auc(x_fpr, tpr_ab[:,i])

    # store predicted probabilities in dictionary
    df_prob.append({
      "dataset": dataset,
      "method": method,  
      "pair": f"{label_a} vs. {label_b}",
      "ytrue_a": ytrue_a, "ytrue_b": ytrue_b,
      "yprob_a": yprob_a, "yprob_b": yprob_b, 
      "tpr_ab": tpr_ab,
      "auc_ab": auc_ab
    })

df_prob = pd.DataFrame(df_prob)
df_prob.head()

In [ ]:
pair_order = [
  "2 vs. (3, 4, 5, 6)",
  "(2, 3) vs. (4, 5, 6)",
  "(2, 3, 4) vs. (5, 6)",
  "(2, 3, 4, 5) vs. 6", 
]

method_color = {
  "FWE": "red", 
  "MSMT": "blue",
  "Original": "green", 
}

fig_width = len(pair_order) * 5
for dataset, df_dataset in df_prob.groupby("dataset"): # for each dataset
  fig, ax = plt.subplots(1, len(pair_order), figsize = (fig_width, 6), tight_layout = True)
  for (pair, method), df_group in df_dataset.groupby(["pair", "method"]): # for each pair and method
    # extract average true positive rates (mean, sem of repeat distribution)
    tpr_ab = df_group["tpr_ab"].values[0]
    y_tpr = tpr_ab.mean(axis = 1)
    s_tpr = tpr_ab.std(axis = 1) / np.sqrt(tpr_ab.shape[1])

    # extract AUC values (mean, sem of repeat distribution)
    auc_ab = df_group["auc_ab"].values[0]
    y_auc = auc_ab.mean()
    s_auc = auc_ab.std() / np.sqrt(auc_ab.shape[-1])

    # determine the figure panel index
    curr_ax = ax[pair_order.index(pair)] # panel index
    curr_ax.plot([0, 1], [0, 1], color = "black", linestyle = "--") # diagonal line
    plot_ribbon(curr_ax, x = x_fpr, y = y_tpr, s = s_tpr, 
                alpha = 0.15, color = method_color[method],
                label = f"{method} (AUC = {y_auc:0.2f})")
    curr_ax.set_xlabel("False Positive Rate")
    curr_ax.set_ylabel("True Positive Rate")
    curr_ax.legend(loc = "lower right")
    curr_ax.set_aspect("equal", "box")
    curr_ax.set_title(pair) # set title
  plt.show()

  save_name = f"figure11_{dataset}_ROC.svg"
  fig.savefig(op.join(paths_save, save_name))